In [1]:
%matplotlib inline
import math
import pandas as pd

In [2]:
df = pd.read_csv('./dataset/transformed_train.csv')
df.describe()

,Yearly_ExpensesK,Avg_Residancy_Altitude,Overall_happiness_score,Will_vote_only_large_party,Financial_agenda_matters,Yearly_IncomeK,Most_Important_Issue,Vote
count,6000.000000,6000.000000,6000.000000,6000.000000,6000.000000,6000.000000,6000.000000,6000.000000
mean,0.002270,-0.006917,0.008513,0.958000,0.186167,0.011338,3.097833,5.108833
std,1.000050,0.988772,0.999945,0.200606,0.389274,0.990657,2.408782,3.360735
min,-2.945120,-2.432355,-3.726302,0.000000,0.000000,-3.918231,0.000000,0.000000
25%,-0.601923,-0.805058,-0.625250,1.000000,0.000000,-0.487121,1.000000,2.000000
50%,-0.349992,-0.234427,-0.058003,1.000000,0.000000,-0.218022,3.000000,5.000000
75%,0.517605,0.606814,0.619365,1.000000,0.000000,0.316993,5.000000,9.000000
max,2.868502,3.344068,2.473634,1.000000,1.000000,3.091690,7.000000,9.000000


In [46]:
def get_votes_count_by_party(df):
    parties = {}
    for party in df.Vote.unique():
        parties[party] = len(df[df.Vote == party])
    return parties;

In [117]:
from sklearn.cluster import KMeans
n_clusters = 100;
cls = KMeans(n_clusters=n_clusters)
res = cls.fit_predict(df.drop('Vote', axis=1))

for v in xrange(n_clusters):
    sum_ = sum(get_votes_count_by_party(df.iloc[[x for x,y in enumerate(res) if y==v]]).itervalues())
    print max({k: (v + .0) / sum_ for k, v in get_votes_count_by_party(df.iloc[[x for x,y in enumerate(res) if y==v]]).iteritems()}.itervalues())

1.0
0.483146067416
0.4
0.360465116279
0.474576271186
0.411214953271
0.628571428571
0.666666666667
0.607329842932
0.878787878788
0.666666666667
0.447368421053
0.607692307692
0.4
0.5
0.338983050847
0.606060606061
0.56338028169
0.350649350649
0.375
0.962962962963
0.966666666667
0.736842105263
0.739130434783
0.46875
0.380952380952
0.631578947368
0.391304347826
0.466666666667
1.0
0.787878787879
0.454022988506
0.642857142857
0.72972972973
0.635944700461
0.666666666667
0.401515151515
0.638888888889
0.928571428571
0.533834586466
1.0
0.875
1.0
0.587301587302
0.823529411765
0.854838709677
0.939393939394
0.576923076923
0.324324324324
0.87323943662
0.454545454545
0.558823529412
1.0
0.454545454545
0.622641509434
0.411764705882
0.428571428571
0.931034482759
0.416666666667
0.460317460317
0.375
0.705882352941
0.381355932203
0.416666666667
0.625
0.8125
1.0
0.875
0.786885245902
1.0
0.48275862069
0.468085106383
0.909090909091
0.613636363636
1.0
0.418604651163
0.782608695652
0.414634146341
0.740740740741


In [104]:
ones = [x for x,y in enumerate(res) if y==1]
zeros = [x for x,y in enumerate(res) if y==0]
f = [x for x,y in enumerate(res) if y==3]

In [105]:
zeros_parties = get_votes_count_by_party(df.iloc[zeros])
f_parties = get_votes_count_by_party(df.iloc[f])
ones_parties = get_votes_count_by_party(df.iloc[ones])
all_parties = get_votes_count_by_party(df)

In [107]:
{k: (v + .0) / sum_ for k, v in f_parties.iteritems()}

{0: 0.2084592145015106,
 1: 0.12386706948640483,
 2: 0.030211480362537766,
 3: 0.07250755287009064,
 4: 0.09667673716012085,
 5: 0.03625377643504532,
 6: 0.015105740181268883,
 7: 0.1419939577039275,
 9: 0.27492447129909364}